# Poke Platform DB Exploration

This notebook connects directly to RDS using Secrets Manager and loads results into pandas DataFrames.

Set `REGION` and `SECRET_ARN` as environment variables if you want to override defaults.


In [2]:
# %pip install pandas
%pip install boto3 pandas psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import sys
from pathlib import Path

# Ensure repo root is on sys.path for local imports
repo_root = Path().resolve().parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

import pandas as pd

from scripts.db_notebook_helpers import fetch_db_secret, read_sql

REGION = os.environ.get("REGION", "us-east-2")
SECRET_ARN = os.environ.get("SECRET_ARN", "arn:aws:secretsmanager:us-east-2:878817878019:secret:PokePlatformStackPostgresSe-uD7oKVjyUmYi-NFBVMe")


## Quick sanity checks


/home/ec2-user/.local/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,"psql:/tmp/query.sql:1: ERROR: syntax error at or near ""AS"""
0,LINE 1: COPY ( SELECT * AS active_tracked FRO...
1,^
2,Exiting session with sessionId: ecs-execute-co...


In [8]:
read_sql("SELECT COUNT(*) AS card_metadata_count FROM card_metadata;", region=REGION, secret_arn=SECRET_ARN)

/home/ec2-user/.local/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,card_metadata_count
0,6131
1,Exiting session with sessionId: ecs-execute-co...


In [5]:
df_cost = read_sql("SELECT * FROM cardmarket_price_snapshot limit 10;", region=REGION, secret_arn=SECRET_ARN)
df_cost.head()

/home/ec2-user/.local/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,snapshot_date,snapshot_ts,asset_id,variant,currency,avg1,avg7,avg30,low_price,trend_price,url,source_updated_at,extra
0,2026-01-19,2026-01-19 13:00:51.662972+00,ptcg:sm6-1,default,EUR,0.12,0.16,0.13,0.02,0.16,https://prices.pokemontcg.io/cardmarket/sm6-1,2026/01/19,"{""avg1"": 0.12, ""avg7"": 0.16, ""avg30"": 0.13, ""l..."
1,2026-01-19,2026-01-19 13:00:51.662972+00,ptcg:sm6-1,reverseHolo,EUR,0.50,0.41,0.33,NaN,0.36,https://prices.pokemontcg.io/cardmarket/sm6-1,2026/01/19,"{""reverseHoloAvg1"": 0.5, ""reverseHoloAvg7"": 0...."
2,2026-01-19,2026-01-19 13:00:51.662972+00,ptcg:sm6-2,default,EUR,0.53,0.58,0.91,0.10,0.26,https://prices.pokemontcg.io/cardmarket/sm6-2,2026/01/19,"{""avg1"": 0.53, ""avg7"": 0.58, ""avg30"": 0.91, ""l..."
3,2026-01-19,2026-01-19 13:00:51.662972+00,ptcg:sm6-2,reverseHolo,EUR,0.35,0.83,0.74,NaN,0.80,https://prices.pokemontcg.io/cardmarket/sm6-2,2026/01/19,"{""reverseHoloAvg1"": 0.35, ""reverseHoloAvg7"": 0..."
4,2026-01-19,2026-01-19 13:00:51.662972+00,ptcg:sm6-2a,default,EUR,1.00,1.73,1.31,0.15,1.93,https://prices.pokemontcg.io/cardmarket/sm6-2a,2026/01/19,"{""avg1"": 1.0, ""avg7"": 1.73, ""avg30"": 1.31, ""lo..."


In [21]:
df_card_metadata = read_sql("SELECT * FROM card_metadata where name = 'Charizard';", region=REGION, secret_arn=SECRET_ARN)
df_card_metadata

/home/ec2-user/.local/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,asset_id,ptcg_card_id,name,set_id,set_name,set_release_date,number,rarity,artist,images_json,raw_json,updated_ts
0,ptcg:pgo-10,pgo-10,Charizard,pgo,Pokémon GO,2022-07-01,10,Rare Holo,N-DESIGN Inc.,"{""large"": ""https://images.pokemontcg.io/pgo/10...","{""hp"": ""170"", ""id"": ""pgo-10"", ""set"": {""id"": ""p...",2026-01-21 13:17:41.471896+00
1,ptcg:xy12-11,xy12-11,Charizard,xy12,Evolutions,2016-11-02,11,Rare Holo,Mitsuhiro Arita,"{""large"": ""https://images.pokemontcg.io/xy12/1...","{""hp"": ""150"", ""id"": ""xy12-11"", ""set"": {""id"": ""...",2026-01-23 13:03:55.417464+00
2,ptcg:sm75-3,sm75-3,Charizard,sm75,Dragon Majesty,2018-09-07,3,Rare Holo,Ryota Murayama,"{""large"": ""https://images.pokemontcg.io/sm75/3...","{""hp"": ""160"", ""id"": ""sm75-3"", ""set"": {""id"": ""s...",2026-01-23 13:15:20.96014+00
3,ptcg:sm9-14,sm9-14,Charizard,sm9,Team Up,2019-02-01,14,Rare,Masakazu Fukuda,"{""large"": ""https://images.pokemontcg.io/sm9/14...","{""hp"": ""150"", ""id"": ""sm9-14"", ""set"": {""id"": ""s...",2026-01-22 13:07:04.980787+00
4,ptcg:det1-5,det1-5,Charizard,det1,Detective Pikachu,2019-04-05,5,Rare Ultra,Framestore,"{""large"": ""https://images.pokemontcg.io/det1/5...","{""hp"": ""180"", ""id"": ""det1-5"", ""set"": {""id"": ""d...",2026-01-22 13:07:41.777894+00
5,ptcg:g1-RC5,g1-RC5,Charizard,g1,Generations,2016-02-22,RC5,Uncommon,Akira Komayama,"{""large"": ""https://images.pokemontcg.io/g1/RC5...","{""hp"": ""160"", ""id"": ""g1-RC5"", ""set"": {""id"": ""g...",2026-01-23 12:56:29.695053+00
6,ptcg:swshp-SWSH066,swshp-SWSH066,Charizard,swshp,SWSH Black Star Promos,2019-11-15,SWSH066,Promo,NC Empire,"{""large"": ""https://images.pokemontcg.io/swshp/...","{""hp"": ""170"", ""id"": ""swshp-SWSH066"", ""set"": {""...",2026-01-21 13:09:25.061605+00
7,ptcg:cel25c-4_A,cel25c-4_A,Charizard,cel25c,Celebrations: Classic Collection,2021-10-08,4,Classic Collection,Mitsuhiro Arita,"{""large"": ""https://images.pokemontcg.io/cel25c...","{""hp"": ""120"", ""id"": ""cel25c-4_A"", ""set"": {""id""...",2026-01-21 13:12:33.443687+00
8,ptcg:swsh4-25,swsh4-25,Charizard,swsh4,Vivid Voltage,2020-11-13,25,Rare,Ryuta Fuse,"{""large"": ""https://images.pokemontcg.io/swsh4/...","{""hp"": ""170"", ""id"": ""swsh4-25"", ""set"": {""id"": ...",2026-01-21 03:46:02.85178+00
9,ptcg:swsh11tg-TG03,swsh11tg-TG03,Charizard,swsh11tg,Lost Origin Trainer Gallery,2022-09-09,TG03,Trainer Gallery Rare Holo,GIDORA,"{""large"": ""https://images.pokemontcg.io/swsh11...","{""hp"": ""170"", ""id"": ""swsh11tg-TG03"", ""set"": {""...",2026-01-21 13:18:15.772437+00


In [4]:
read_sql("SELECT COUNT(*) AS card_metadata_count FROM card_metadata;", region=REGION, secret_arn=SECRET_ARN)


/home/ec2-user/.local/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,card_metadata_count
0,6131
1,Exiting session with sessionId: ecs-execute-co...


In [5]:
read_sql("SELECT MAX(updated_ts) AS latest_updated_ts FROM card_metadata;", region=REGION, secret_arn=SECRET_ARN)


/home/ec2-user/.local/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,latest_updated_ts
0,2026-01-23 13:16:31.781371+00
1,Exiting session with sessionId: ecs-execute-co...


## Sample slice from card_metadata


In [6]:
df_cards = read_sql("SELECT asset_id, name, set_name, rarity, updated_ts FROM card_metadata ORDER BY updated_ts DESC LIMIT 50;", region=REGION, secret_arn=SECRET_ARN)
df_cards.head(10)


/home/ec2-user/.local/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,asset_id,name,set_name,rarity,updated_ts
0,ptcg:sm8-32,Vespiquen,Lost Thunder,Uncommon,2026-01-23 13:16:31.781371+00
1,ptcg:sm8-51,Pyroar,Lost Thunder,Rare,2026-01-23 13:16:31.781371+00
2,ptcg:sm8-16,Shuckle,Lost Thunder,Uncommon,2026-01-23 13:16:31.781371+00
3,ptcg:sm8-31,Combee,Lost Thunder,Common,2026-01-23 13:16:31.781371+00
4,ptcg:sm8-41,Quilava,Lost Thunder,Uncommon,2026-01-23 13:16:31.781371+00
5,ptcg:sm8-50,Litleo,Lost Thunder,Common,2026-01-23 13:16:31.781371+00
6,ptcg:sm8-8,Meganium,Lost Thunder,Rare Holo,2026-01-23 13:16:31.781371+00
7,ptcg:sm8-15,Pineco,Lost Thunder,Common,2026-01-23 13:16:31.781371+00
8,ptcg:sm8-23,Wurmple,Lost Thunder,Common,2026-01-23 13:16:31.781371+00
9,ptcg:sm8-30,Ninjask,Lost Thunder,Uncommon,2026-01-23 13:16:31.781371+00


In [9]:
df_car2= read_sql("SELECT * from cards limit 10;", region=REGION, secret_arn=SECRET_ARN)
df_car2.head(10)

/home/ec2-user/.local/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,"psql:/tmp/query.sql:1: ERROR: relation ""cards"" does not exist"
0,LINE 1: COPY ( SELECT * from cards limit 10 )...
1,^
2,Exiting session with sessionId: ecs-execute-co...


In [ ]:
tables

## Your custom query


In [12]:
# asset_id - unique identifier
custom_sql = """
SELECT count(*)
FROM card_metadata
"""
df_custom = read_sql(custom_sql, region=REGION, secret_arn=SECRET_ARN)
df_custom


/home/ec2-user/.local/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,count
0,6131
1,Exiting session with sessionId: ecs-execute-co...


In [ ]:
# asset_id - unique identifier
custom_sql = """
SELECT *
FROM card_metadata
limit 5
"""
df_custom1 = read_sql(custom_sql, region=REGION, secret_arn=SECRET_ARN)
df_custom1
# check what is inside raw_json

/home/ec2-user/.local/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,asset_id,ptcg_card_id,name,set_id,set_name,set_release_date,number,rarity,artist,images_json,raw_json,updated_ts
0,ptcg:swsh12-1,swsh12-1,Venonat,swsh12,Silver Tempest,2022-11-11,1.0,Common,Sekio,"{""large"": ""https://images.pokemontcg.io/swsh12...","{""hp"": ""60"", ""id"": ""swsh12-1"", ""set"": {""id"": ""...",2026-01-24 14:13:53.359936+00
1,ptcg:swsh12-2,swsh12-2,Venomoth,swsh12,Silver Tempest,2022-11-11,2.0,Uncommon,Kagemaru Himeno,"{""large"": ""https://images.pokemontcg.io/swsh12...","{""hp"": ""90"", ""id"": ""swsh12-2"", ""set"": {""id"": ""...",2026-01-24 14:13:53.359936+00
2,ptcg:swsh12-3,swsh12-3,Spinarak,swsh12,Silver Tempest,2022-11-11,3.0,Common,Pani Kobayashi,"{""large"": ""https://images.pokemontcg.io/swsh12...","{""hp"": ""60"", ""id"": ""swsh12-3"", ""set"": {""id"": ""...",2026-01-24 14:13:53.359936+00
3,ptcg:swsh9-2,swsh9-2,Exeggutor,swsh9,Brilliant Stars,2022-02-25,2.0,Uncommon,Naoyo Kimura,"{""large"": ""https://images.pokemontcg.io/swsh9/...","{""hp"": ""140"", ""id"": ""swsh9-2"", ""set"": {""id"": ""...",2026-01-24 14:01:08.699741+00
4,ptcg:swsh12-4,swsh12-4,Ariados,swsh12,Silver Tempest,2022-11-11,4.0,Rare Holo,Tika Matsuno,"{""large"": ""https://images.pokemontcg.io/swsh12...","{""hp"": ""90"", ""id"": ""swsh12-4"", ""set"": {""id"": ""...",2026-01-24 14:13:53.359936+00
5,Exiting session with sessionId: ecs-execute-co...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# asset_id - unique identifier
# SELECT *
# FROM cardmarket_price_snapshot
# WHERE name ILIKE '%Vaporeon%'
custom_sql = """
SELECT * FROM valuation_daily limit 10;
"""
df_custom2 = read_sql(custom_sql, region=REGION, secret_arn=SECRET_ARN)
df_custom2

/home/ec2-user/.local/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,val_date,asset_id,market_price,smooth_price,forecast_price,gap,gap_pct,confidence,rationale_json,strategy_name,strategy_version,run_id,ts_created
0,Exiting session with sessionId: ecs-execute-command-lr7bxx9h2tiagackg6ildqeary.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# widen display for all columns
import pandas as pd
pd.set_option("display.max_colwidth", None)
df_custom2[["raw_json", "images_json"]].head(2)


,raw_json,images_json
0,"{""hp"": ""60"", ""id"": ""swsh12-1"", ""set"": {""id"": ""swsh12"", ""name"": ""Silver Tempest"", ""total"": 215, ""images"": {""logo"": ""https://images.pokemontcg.io/swsh12/logo.png"", ""symbol"": ""https://images.pokemontcg.io/swsh12/symbol.png""}, ""series"": ""Sword & Shield"", ""ptcgoCode"": ""SIT"", ""updatedAt"": ""2022/09/09 11:45:00"", ""legalities"": {""expanded"": ""Legal"", ""standard"": ""Legal"", ""unlimited"": ""Legal""}, ""releaseDate"": ""2022/11/11"", ""printedTotal"": 195}, ""name"": ""Venonat"", ""types"": [""Grass""], ""artist"": ""Sekio"", ""images"": {""large"": ""https://images.pokemontcg.io/swsh12/1_hires.png"", ""small"": ""https://images.pokemontcg.io/swsh12/1.png""}, ""number"": ""1"", ""rarity"": ""Common"", ""attacks"": [{""cost"": [""Grass""], ""name"": ""Tackle"", ""text"": """", ""damage"": ""20"", ""convertedEnergyCost"": 1}], ""subtypes"": [""Basic""], ""evolvesTo"": [""Venomoth""], ""supertype"": ""Pokémon"", ""tcgplayer"": {""url"": ""https://prices.pokemontcg.io/tcgplayer/swsh12-1"", ""prices"": {""normal"": {""low"": 0.01, ""mid"": 0.1, ""high"": 20.0, ""market"": 0.08, ""directLow"": null}, ""reverseHolofoil"": {""low"": 0.05, ""mid"": 0.2, ""high"": 2.35, ""market"": 0.15, ""directLow"": null}}, ""updatedAt"": ""2026/01/24""}, ""cardmarket"": {""url"": ""https://prices.pokemontcg.io/cardmarket/swsh12-1"", ""prices"": {""avg1"": 0.15, ""avg7"": 0.04, ""avg30"": 0.05, ""lowPrice"": 0.02, ""trendPrice"": 0.05, ""germanProLow"": 0.0, ""lowPriceExPlus"": 0.02, ""reverseHoloLow"": 0.02, ""suggestedPrice"": 0.0, ""reverseHoloAvg1"": 0.2, ""reverseHoloAvg7"": 0.15, ""reverseHoloSell"": 0.17, ""averageSellPrice"": 0.05, ""reverseHoloAvg30"": 0.15, ""reverseHoloTrend"": 0.14}, ""updatedAt"": ""2026/01/24""}, ""flavorText"": ""Lives in the shadows of tall trees where it eats bugs. It is attracted by light at night."", ""legalities"": {""expanded"": ""Legal"", ""standard"": ""Legal"", ""unlimited"": ""Legal""}, ""weaknesses"": [{""type"": ""Fire"", ""value"": ""×2""}], ""retreatCost"": [""Colorless""], ""regulationMark"": ""F"", ""convertedRetreatCost"": 1, ""nationalPokedexNumbers"": [48]}","{""large"": ""https://images.pokemontcg.io/swsh12/1_hires.png"", ""small"": ""https://images.pokemontcg.io/swsh12/1.png""}"
1,"{""hp"": ""90"", ""id"": ""swsh12-2"", ""set"": {""id"": ""swsh12"", ""name"": ""Silver Tempest"", ""total"": 215, ""images"": {""logo"": ""https://images.pokemontcg.io/swsh12/logo.png"", ""symbol"": ""https://images.pokemontcg.io/swsh12/symbol.png""}, ""series"": ""Sword & Shield"", ""ptcgoCode"": ""SIT"", ""updatedAt"": ""2022/09/09 11:45:00"", ""legalities"": {""expanded"": ""Legal"", ""standard"": ""Legal"", ""unlimited"": ""Legal""}, ""releaseDate"": ""2022/11/11"", ""printedTotal"": 195}, ""name"": ""Venomoth"", ""types"": [""Grass""], ""artist"": ""Kagemaru Himeno"", ""images"": {""large"": ""https://images.pokemontcg.io/swsh12/2_hires.png"", ""small"": ""https://images.pokemontcg.io/swsh12/2.png""}, ""number"": ""2"", ""rarity"": ""Uncommon"", ""attacks"": [{""cost"": [""Grass""], ""name"": ""Miracle Powder"", ""text"": ""Flip a coin. If heads, choose a Special Condition. Your opponent's Active Pokémon is now affected by that Special Condition."", ""damage"": ""30"", ""convertedEnergyCost"": 1}, {""cost"": [""Grass"", ""Colorless""], ""name"": ""Gust"", ""text"": """", ""damage"": ""70"", ""convertedEnergyCost"": 2}], ""subtypes"": [""Stage 1""], ""supertype"": ""Pokémon"", ""tcgplayer"": {""url"": ""https://prices.pokemontcg.io/tcgplayer/swsh12-2"", ""prices"": {""normal"": {""low"": 0.01, ""mid"": 0.1, ""high"": 19.89, ""market"": 0.1, ""directLow"": 0.05}, ""reverseHolofoil"": {""low"": 0.03, ""mid"": 0.2, ""high"": 2.39, ""market"": 0.16, ""directLow"": null}}, ""updatedAt"": ""2026/01/24""}, ""cardmarket"": {""url"": ""https://prices.pokemontcg.io/cardmarket/swsh12-2"", ""prices"": {""avg1"": 0.08, ""avg7"": 0.07, ""avg30"": 0.05, ""lowPrice"": 0

In [18]:
# asset_id - unique identifier
custom_sql = """
SELECT 
  DATE(updated_ts) AS load_date,
  COUNT(*) AS cards_loaded
FROM card_metadata
group by DATE(updated_ts)
ORDER BY load_date;
"""
df_custom2 = read_sql(custom_sql, region=REGION, secret_arn=SECRET_ARN)
df_custom2

/home/ec2-user/.local/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,load_date,cards_loaded
0,2026-01-21,200.0
1,2026-01-23,100.0
2,2026-01-24,5831.0
3,Exiting session with sessionId: ecs-execute-command-uo46qquxbnq3pa2v8ltb2eeeh4.,NaN


In [19]:
# asset_id - unique identifier
custom_sql = """
SELECT asset_id, variant, market, low, mid, high, snapshot_date
FROM tcgplayer_price_snapshot
ORDER BY snapshot_date DESC LIMIT 50;
"""
df_custom2 = read_sql(custom_sql, region=REGION, secret_arn=SECRET_ARN)
df_custom2

/home/ec2-user/.local/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,asset_id,variant,market,low,mid,high,snapshot_date
0,ptcg:zsv10pt5-99,holofoil,9.15,7.50,9.76,40.00,2026-01-24
1,ptcg:zsv10pt5-97,holofoil,8.71,6.98,9.32,30.00,2026-01-24
2,ptcg:zsv10pt5-96,holofoil,27.32,22.00,29.90,65.43,2026-01-24
3,ptcg:zsv10pt5-93,holofoil,2.47,2.00,3.27,16.00,2026-01-24
4,ptcg:zsv10pt5-92,holofoil,24.47,21.51,27.00,49.99,2026-01-24
5,ptcg:zsv10pt5-90,holofoil,5.96,5.05,6.79,40.00,2026-01-24
6,ptcg:zsv10pt5-9,reverseHolofoil,0.09,0.01,0.15,2.37,2026-01-24
7,ptcg:zsv10pt5-9,normal,0.03,0.01,0.10,1.53,2026-01-24
8,ptcg:zsv10pt5-88,holofoil,11.31,10.15,12.56,24.99,2026-01-24
9,ptcg:zsv10pt5-86,reverseHolofoil,0.23,0.02,0.29,4.95,2026-01-24


In [29]:
# asset_id - unique identifier
custom_sql = """
            WITH latest AS (SELECT MAX(val_date) AS val_date FROM valuation_daily)
            SELECT vd.val_date, vd.asset_id, vd.market_price, vd.forecast_price, vd.gap_pct,
            vd.confidence, cm.name, cm.set_name, cm.rarity
            FROM valuation_daily vd
            JOIN latest l ON vd.val_date = l.val_date
            LEFT JOIN card_metadata cm ON cm.asset_id = vd.asset_id 
            ORDER BY vd.gap_pct DESC LIMIT 50;
"""
df_custom2 = read_sql(custom_sql, region=REGION, secret_arn=SECRET_ARN)
df_custom2

/home/ec2-user/.local/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,val_date,asset_id,market_price,forecast_price,gap_pct,confidence,name,set_name,rarity
0,2026-01-24,ptcg:sm4-77,0.45,0.607379,0.349730,1.0,Kommo-o,Crimson Invasion,Rare
1,2026-01-24,ptcg:sm115-24,0.84,0.948257,0.128878,1.0,Zapdos,Hidden Fates,Rare Holo
2,2026-01-24,ptcg:swsh10-59,0.10,0.111837,0.118374,1.0,Mismagius,Astral Radiance,Rare
3,2026-01-24,ptcg:me2-53,0.14,0.156425,0.117321,1.0,Flygon,Phantasmal Flames,Rare
4,2026-01-24,ptcg:sv8pt5-65,0.11,0.122288,0.111709,1.0,Roaring Moon,Prismatic Evolutions,Rare
5,2026-01-24,ptcg:sv10-20,0.22,0.243306,0.105937,1.0,Team Rocket's Spidops,Destined Rivals,Rare
6,2026-01-24,ptcg:swsh12-23,0.09,0.099339,0.103765,1.0,Victini,Silver Tempest,Rare
7,2026-01-24,ptcg:sv2-97,0.27,0.297402,0.101490,1.0,Mimikyu,Paldea Evolved,Rare
8,2026-01-24,ptcg:sv10-49,0.47,0.517473,0.101006,1.0,Misty's Gyarados,Destined Rivals,Rare
9,2026-01-24,ptcg:sv8pt5-13,0.23,0.253101,0.100441,1.0,Flareon,Prismatic Evolutions,Rare


In [30]:
# asset_id - unique identifier
custom_sql = """
    SELECT MAX(snapshot_date) AS latest_snapshot_date FROM card_metadata;
"""
df_custom2 = read_sql(custom_sql, region=REGION, secret_arn=SECRET_ARN)
df_custom2


/home/ec2-user/.local/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,latest_snapshot_date
0,2026-01-24
1,Exiting session with sessionId: ecs-execute-command-lyysvzp2ddy9c6qorlsojk3u7y.
